In [9]:
import cost_model
import os
import json
import sys

from pprint import pprint
from IPython.display import SVG, display, HTML

def show_svg(url1, url2):
    display(HTML("""<table><tr><td><img style='width:100px' src='{}'/></td>
                               <td><img style='width:100px' src='{}'/></td></tr></table>""".format(url1, url2)))

# the folder containing all exports
export_dir = "./saved_designs"

def load_design(design_dir):
    """the directory containing exported designs"""

    design = {}
    # load saved designs and trashed designs
    with open(os.path.join(design_dir, "saved.json")) as f:
        design["saved"] = json.load(f)["saved"]
    with open(os.path.join(design_dir, "trashed.json")) as f:
        design["trashed"] = json.load(f)["trashed"]

    # load paths for saved svgs, relates to saved designs by their canvas ids
    svg_dir = os.path.join(design_dir, "saved_svgs")
    if not os.path.isdir(svg_dir):
        svg_dir = os.path.join(design_dir, "import_these_into_xd")
    design["svg_paths"] = {}
    for fname in os.listdir(svg_dir):
        if fname.endswith(".svg"):
            design_id = fname.split(".")[0].split("-")[-1]
            design["svg_paths"][design_id] = os.path.join(svg_dir, fname)

    design["dirname"] = design_dir
    return design

def analyze_design(all_designs):
    """ Analyze design scores from all saved apps, the scores are normalized across all designs
        For each pair of saved designs in each file, display their diversity score as well as svgs
        Inputs:
            all_designs: a list of saved designs (saved.json)
    """
    
    raw_diversity_score = {}
    
    for exp_id, design in all_designs.items():   
        
        if len(design["saved"]) == 1:
            print("[error] this directory contains only 1 saved desigsn.")
            continue

        for i in range(len(design["saved"])):
            for j in range(i + 1, len(design["saved"])):
                di, dj = design["saved"][i], design["saved"][j]
                #diversity_score, diff = cost_model.compute_diversity_score(di["elements"], dj["elements"])
                raw_diversity_score[(exp_id, i, j)] = cost_model.compute_unnormalized_diversity_score(
                    di["elements"], dj["elements"])

    final_scores = cost_model.normalize_diversity_scores(raw_diversity_score)
    
    design_pairs = []
    for pair_id in final_scores:
        exp_id, i,j = pair_id
        score = (final_scores[pair_id]["alt_group_score"]
                    + final_scores[pair_id]["pos_diff_score"] 
                    + final_scores[pair_id]["rel_dist_diff_score"] 
                    + 2 * final_scores[pair_id]["size_diff_score"])
        design_pair = {
            "exp_id": exp_id,
            "score": score,
            "score_detail": final_scores[pair_id],
            "di": all_designs[exp_id]["saved"][i],
            "dj": all_designs[exp_id]["saved"][j],
        }
        
        design_pairs.append(design_pair)

    design_pairs.sort(key=lambda x: x['score'], reverse=True)
    for pair in design_pairs: 
        no_svg_to_display = False
        exp_id = pair["exp_id"]
        di = pair['di']
        dj = pair['dj']
        for d in [di, dj]:
            if d["id"] not in all_designs[exp_id]["svg_paths"]:
                #print("  [Error] no svg for id {}".format(d["id"]))
                no_svg_to_display = True

        if no_svg_to_display:
            #print("  " + design["dirname"])
            continue
        score = pair['score']
        score_detail = pair['score_detail']
        print("# experiments from {}".format(pair["exp_id"]))
        print("# diversity score {}".format(score))
        print("# score details")
        pprint(score_detail)

        svg_i = all_designs[exp_id]["svg_paths"][di["id"]]
        svg_j = all_designs[exp_id]["svg_paths"][dj["id"]]
        show_svg(svg_i, svg_j)
        print("")
            
# iterate over all exported deisngs
all_designs = {}
for dirname in os.listdir(export_dir):
    design_dir = os.path.join(export_dir, dirname)
    if not os.path.isdir(design_dir):
        continue
    design = load_design(design_dir)
    all_designs[dirname] = design
    
analyze_design(all_designs)

[error] this directory contains only 1 saved desigsn.
# experiments from exported_from_scout (88)
# diversity score 1.649860340612943
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.574876725632672,
 'rel_dist_diff_score': 0.3888477791167961,
 'size_diff_score': 0.34306791793173746}


,



# experiments from exported_from_scout (86)
# diversity score 1.6395432518580997
# score details
{'alt_group_score': 0.08333333333333333,
 'pos_diff_score': 0.5988578094755432,
 'rel_dist_diff_score': 0.3644425601965499,
 'size_diff_score': 0.2964547744263366}


,



# experiments from exported_from_scout (88)
# diversity score 1.625552110596903
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.6310314135841235,
 'rel_dist_diff_score': 0.3883471296558854,
 'size_diff_score': 0.30308678367844716}


...

,



# experiments from exported_from_scout (27)
# diversity score 0.3474764055755152
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.09559726853286589,
 'rel_dist_diff_score': 0.07665997603003367,
 'size_diff_score': 0.0876095805063078}


,



# experiments from exported_from_scout (97)
# diversity score 0.3249338422902093
# score details
{'alt_group_score': 0.0,
 'pos_diff_score': 0.08268761514429405,
 'rel_dist_diff_score': 0.053735527472005364,
 'size_diff_score': 0.09425534983695494}


,
